In [70]:
from argparse import ArgumentParser
from fastprogress.fastprogress import master_bar, progress_bar
import torch
import pandas as pd
import os
import numpy as np

In [71]:
def train_one_epoch(dataloader, model, criterion, optimizer, device, mb):

    # Put the model into training mode
    model.train()

    # Loop over the data using the progress_bar utility
    for _, (X, Y) in progress_bar(DataLoaderProgress(dataloader), parent=mb):
        X, Y = X.to(device), Y.to(device)

        # Compute model output and then loss
        output = model(X)
        loss = criterion(output, Y)

        # - zero-out gradients
        optimizer.zero_grad()
        # - compute new gradients
        loss.backward()
        # - update paramters
        optimizer.step()

In [72]:
def validate(dataloader, model, criterion, device, epoch, num_epochs, mb):

    # Put the model into validation/evaluation mode
    model.eval()

    N = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    loss, num_correct = 0, 0

    # Tell pytorch to stop updating gradients when executing the following
    with torch.no_grad():

        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device)

            # Compute the model output
            output = model(X)

            # - compute loss
            loss += criterion(output, Y).item()
            # - compute the number of correctly classified examples
            num_correct += (output.argmax(1) == Y).type(torch.float).sum().item()

        loss /= num_batches
        accuracy = num_correct / N

    message = "Initial" if epoch == 0 else f"Epoch {epoch:>2}/{num_epochs}:"
    message += f" accuracy={100*accuracy:5.2f}%"
    message += f" and loss={loss:.3f}"
    mb.write(message)

In [73]:
def train(model, criterion, optimizer, train_loader, valid_loader, device, num_epochs):

    mb = master_bar(range(num_epochs))

    validate(valid_loader, model, criterion, device, 0, num_epochs, mb)

    for epoch in mb:
        train_one_epoch(train_loader, model, criterion, optimizer, device, mb)
        validate(valid_loader, model, criterion, device, epoch + 1, num_epochs, mb)

In [74]:
def main():
    aparser = ArgumentParser("FIFAI--Train a neural network to predict EPL scorelines.")
    aparser.add_argument("epl_data", type=str, help="Path to store/find the EPL games dataset")
    aparser.add_argument("--num_epochs", type=int, default=10)
    aparser.add_argument("--batch_size", type=int, default=128)
    aparser.add_argument("--learning_rate", type=float, default=0.01)
    aparser.add_argument("--momentum", type=float, default=0.9)
    aparser.add_argument("--gpu", action="store_true")

    args = aparser.parse_args()

    # Use GPU if requested and available
    device = "cuda" if args.gpu and torch.cuda.is_available() else "cpu"
    

    # Get data loaders
    # train_loader, valid_loader = get_epl_data_loaders(args.epl_data, args.batch_size, 0)
    
    
#     model = torch.nn.Sequential(torch.nn.Flatten(), torch.nn.Linear(in_features=?, out_features=10),)

    # - create a CrossEntropyLoss criterion
    # - create an SGD optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=args.learning_rate, momentum=args.momentum)

    train(model, criterion, optimizer, train_loader, valid_loader, device, args.num_epochs)

In [75]:
# creates a model for each team file in the 'teams_five_season_data' folder
def load_data_into_models():
    path = "../teams_five_season_data"
    
    team_models = []

    for file in os.listdir(path):
        model = Team(path+"/"+file)
        team_models.append(model)
    
    return team_models

In [80]:
t1 = Team("../teams_five_season_data/Arsenal16_21.csv")
t2 = Team("../teams_five_season_data/Burnley16_21.csv")

dl = torch.utils.data.DataLoader(t1, batch_size = 10)

In [79]:
class Team ():
    def __init__(self, file_path):
        file_out = pd.read_csv(file_path)
        # x_axis_labels = file_out.iloc[0, 1:8].values
        
        data = file_out.iloc[1:191, 1:8].values
        x = []
        y = []
        
        for match in data:
            # convert team names to int encoding
            match[0] = encode_string_as_int(match[0])
            match[1] = encode_string_as_int(match[1])
            
            # create lists for input and target data
            x.append(np.delete(match, 4).tolist())
            y.append(match[4])
            
        # cast all string input data to ints
        for match in x:
            for attr in range(0,6):
                match[attr] = int(match[attr])
                
        # cast all string target data to ints 
        for match_result in range(len(y)):
            y[match_result] = int(y[match_result])
    
        
        # print("training data:")
        # print(x)
        # print()
        
        # print("match results (aka targets):")
        # print(y)
        
        self.X_train = torch.tensor(x, dtype = torch.double)
        self.Y_train = torch.tensor(y, dtype = torch.double)
        
        # print (self.X_train)
        # print (self.Y_train)
    
    def __len__(self):
        return len(self.Y_train)
    
    def __getitem__(self, idx):
        return self.X_train[idx], self.Y_train[idx]
        

In [78]:
# converts team names into readable integer encodings
def encode_string_as_int(string):
    ret = ""
    for c in string:
        ret += str(ord(c))
    return int(ret)